In [180]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [243]:
df = pd.read_csv('Twitter_Guncel.csv')

In [246]:
# Clean Tweet Function
def clean_tweet(tweet):
    tweet = re.sub(r'@[A-Za-z0-9]+', ' ', tweet) # Removing @mentions
    tweet = re.sub(r'#', '', tweet) # Removing '#' hash tag
    tweet = re.sub(r'RT[\s]+', '', tweet) # Removing RT
    tweet = re.sub(r'https?:\/\/\S+', '', tweet) # Removing hyperlink
    tweet = re.sub(r'&amp;', ' ', tweet) # Removing &amp;
    tweet = re.sub(r'&gt;', ' ', tweet) # Removing &gt;
    tweet = re.sub(r'&lt;', ' ', tweet) # Removing &lt;
    tweet = re.sub(r'&quot;', ' ', tweet) # Removing &quot;
    tweet = re.sub(r'&apos;', ' ', tweet) # Removing &apos;
    tweet = re.sub(r'&nbsp;', ' ', tweet) # Removing &nbsp;
    tweet = re.sub(r'&iexcl;', ' ', tweet) # Removing &iexcl;
    # clean links such as https://t.co/Tu3oE2zQ1Q
    tweet = re.sub(r'https://t.co/[A-Za-z0-9]+', ' ', tweet)
    return tweet

    
# apply on column
df['contents'] = df['contents'].apply(clean_tweet)
# clean \n from contents
df['contents'] = df['contents'].str.replace(r'\n', ' ')
# replace / with space
df['contents'] = df['contents'].str.replace(r'/', ' ')
# replace \ with space
df['contents'] = df['contents'].str.replace(r'\\', ' ')

df['contents'] = df['contents'].str.lower()

In [248]:
# replace , with space
df['contents'] = df['contents'].str.replace(r',', ' ')

In [249]:
# rename contents as adres
df.rename(columns={'contents':'adres'}, inplace=True)

In [250]:
qf = pd.read_excel('sehıler.xlsx')
qf.columns= ['1','il','ilce']
city = ['adana', 'adıyaman', 'batman', 'bingöl', 'bingol','bitlis', 'diyarbakır', 'elazığ', 'gaziantep', 'hakkari', 'hatay', 'kahramanmaraş', 'kilis', 'malatya', 'mardin', 'mersin', 'osmaniye', 'şanlıurfa','adiyaman','sanliurfa']
qf['il'] = qf['il'].str.lower()
check = qf[qf['il'].isin(city)]['ilce'].unique().tolist()
check = [word.lower() for word in check]
#  find items that is present in check but not in city
# x = [item for item in check if item not in city]

In [251]:
# renmae contents as adres
df.rename(columns={'contents':'adres'}, inplace=True)

In [252]:
# search citys such as gaziantep adana malatya and put all into another column use join
df['deprem'] = df['adres'].str.findall('|'.join(city)).astype(str)
df['deprem'] = df['deprem'].str.replace('[','').str.replace(']','')

In [188]:
il = qf['il'].unique().tolist()
iller = list(set(il) - set(city))
# Deprem olmayan iller

In [190]:
# Yardım gelen iller, diğer iller.
df['Gelen'] = df['adres'].str.findall('|'.join(iller)).astype(str)
df['Gelen'] = df['Gelen'].str.replace('[','').str.replace(']','')

In [191]:
isinma_keywords = ["soguktan","usuyoruz","donmak","soguk","soguk deprem","hava soguk yardim edin",
                   "hipotermi","deprem battaniye","isinmak istiyoruz","donacagiz","deprem eldiven",
                   "deprem kaban","deprem mont","deprem tup","deprem cadir","cadir istiyoruz",
                   "soguktan olecegiz","soguktan olecegiz yardim edin","isinacak yerimiz yok",
                   "isinmaliyiz","kalacak yer","kalacak yerimiz yok","titriyoruz","deprem titriyoruz",
                   "acil soguktan","acil kalacak yer","acil isinma yardim","acil battaniye","acil kiyafet",
                   "acil yatak"]

barinma_keywords = ["barinma yardim","cadir yardim","cadir","cadir istiyoruz","soguk","soguktan olecegiz",
                    "kalacak yer","usuyoruz","donuyoruz","olmek uzereyiz","isinmak istiyoruz","soguktan olmek",
                    "hipotermi","siginma alani","cadir talebi","barinma talebi","cadirimiz yok","cocuklar usuyor",
                    "ailem usuyor","disaridayiz","hava soguk","soguk","soguktan usuyoruz","evim yok","evimiz yok",
                    "cadir bulamiyoruz","aracta kaliyoruz","arabada kaliyoruz","sokaktayiz"
                    "soguktan","usuyoruz","donmak","soguk deprem","hava soguk yardim edin",
                    "hipotermi","deprem battaniye","isinmak istiyoruz","donacagiz","deprem eldiven",
                    "deprem kaban","deprem mont","deprem tup","deprem cadir","cadir istiyoruz","donarak",
                    "soguktan olecegiz yardim edin","isinacak yerimiz yok",
                    "isinmaliyiz","kalacak yerimiz yok","titriyoruz","deprem titriyoruz",
                    "acil soguktan","acil kalacak yer","acil isinma yardim","acil battaniye","acil kiyafet",
                    "acil yatak","soğuk","kalacak yer"]

barınma_keywords = ["barinma","barinma istiyoruz", "acil barinma yardim edin", "çadır", "konteyner", "ailem usuyor","soguktan olecegiz","çadır","disaridayiz","arabada kaliyoruz"] 

yemek_keywords = ["yemek", "yemek istiyoruz", "yemek yardim edin", "acil yemek yardim edin","gıda","konserve","açlık","açlıktan","açlıktan öleceğiz","açız","erzak","süt"]

su_keywords = ["su", "su istiyoruz", "su yardim edin", "acil su yardim edin","su lazım, sıvı lazım"]

yagma = [ "suriyeliler",'çaldılar' ,'talan', 'yagma', "yağmalandı","saldırıyor","yağma","gasp", "kapışılıyor", "hırsızlık", "çalma", "soygun","tır yağmalanıyor","tıra saldırıyorlar", "yağmalanıyor","kaos"]

tuvalet_keywords = ["tuvalet", "tuvalet"]

giysi_keywords = ["giysi talebi", "giysi", "battaniye", "yagmurluk", "kazak", "corap", "soguk", "bot", "isitici", "cadir", "hava", 
                    "camasir", "pijama", "soguktan", "yatak", "sisme", "bez", "bezi", "bebek bezi", "soba", 
                    "hijyen", "temizlik", "temizlik malzemesi", "basortu", "hijyen paketi", "kar", "hipotermi", "donmak", "yorgan"]

ihtiyaclist = ["yardım gitmemiş", "ihtiyac", "ihtiyacı","ihtiyaci","bez", "gıda", "i̇htiyaç", "konserve", "çocuk  bezi", "maması", "orkid", "battaniye", "çocuk", "ilaçları", "calpol", "dolven", "ağrı", "kesici", "eldiven", "bere", "atkı", "bot", "mont" ,"çorap"]

In [193]:
import warnings
warnings.filterwarnings('ignore')

df['isinma'] = df['adres'].str.findall('|'.join(isinma_keywords)).astype(str)
df['isinma'] = df['isinma'].str.replace('[','').str.replace(']','')

df['barinma'] = df['adres'].str.findall('|'.join(barinma_keywords)).astype(str)
df['barinma'] = df['barinma'].str.replace('[','').str.replace(']','')

df['yemek'] = df['adres'].str.findall('|'.join(yemek_keywords)).astype(str)
df['yemek'] = df['yemek'].str.replace('[','').str.replace(']','')

df['su'] = df['adres'].str.findall('|'.join(su_keywords)).astype(str)
df['su'] = df['su'].str.replace('[','').str.replace(']','')

df['tuvalet'] = df['adres'].str.findall('|'.join(tuvalet_keywords)).astype(str)
df['tuvalet'] = df['tuvalet'].str.replace('[','').str.replace(']','')

df['giysi'] = df['adres'].str.findall('|'.join(giysi_keywords)).astype(str)
df['giysi'] = df['giysi'].str.replace('[','').str.replace(']','')

df['yagma'] = df['adres'].str.findall('|'.join(yagma)).astype(str)
df['yagma'] = df['yagma'].str.replace('[','').str.replace(']','')

In [194]:
# find sinma	barinma	yemek su tuvalet giysi yagma any of them not empty
qf = df.loc[(df['isinma'] != '') | (df['barinma'] != '') | (df['yemek'] != '') | (df['su'] != '') | (df['tuvalet'] != '') | (df['giysi'] != '') | (df['yagma'] != '')]

In [195]:
from datetime import datetime, timedelta
# rename createdAt as date
qf = qf.rename(columns={'createdAt': 'date'})

In [196]:
# Convert the date strings to datetime objects
qf['date'] = qf['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))

# Add three hours to each datetime object
qf['date'] = qf['date'].apply(lambda x: x + timedelta(hours=3))

# Format the datetime objects as desired
qf['date'] = qf['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# sort qf by deprem	Gelen	isinma	barinma	yemek	su	tuvalet	giysi yagma
qf.sort_values(by=['date','isinma','barinma','yemek','su','tuvalet','giysi','yagma'], ascending=False).head(20)

In [219]:
cleaner = ['isinma','barinma','yemek','su','tuvalet','giysi','yagma', 'deprem', 'Gelen']
for col in cleaner:
    qf[col] = qf[col].str.replace("'","")

In [228]:
qf = qf[qf['giysi'] != 'kar']

In [230]:
# sum word counts in this columns deprem	Gelen	isinma	barinma	yemek	su	tuvalet	giysi	yagma
# word count function on column
def word_count(x):
    return len(x.split())

# word count on each row
qf['isinma_count'] = qf['isinma'].apply(word_count)
qf['barinma_count'] = qf['barinma'].apply(word_count)
qf['yemek_count'] = qf['yemek'].apply(word_count)
qf['su_count'] = qf['su'].apply(word_count)
qf['tuvalet_count'] = qf['tuvalet'].apply(word_count)
qf['giysi_count'] = qf['giysi'].apply(word_count)
qf['yagma_count'] = qf['yagma'].apply(word_count)

# total
qf['total'] = qf['isinma_count'] + qf['barinma_count'] + qf['yemek_count'] + qf['su_count'] + qf['tuvalet_count'] + qf['giysi_count'] + qf['yagma_count']

In [ ]:
# pd max colwidth
pd.set_option('display.max_colwidth', 1)
# pd show max rows
pd.set_option('display.max_rows', 11)
qf.sort_values('total', ascending=False).head(200)

In [238]:
qf = qf[qf['total'] > 2 ]

In [239]:
# clean columns that contains count in them
qf = qf.drop(['isinma_count','barinma_count','yemek_count','su_count','tuvalet_count','giysi_count','yagma_count'], axis=1)

In [241]:
qf.to_excel('Yardım_Talepleri.xlsx', index=False)

In [ ]:
qf.to_excel('İhtiyaclar.xlsx', index=False)